# Env

In [ ]:
%pip install -q langchain langchain_community langchain_chroma langchain_experimental langchain-text-splitters
%pip install -q langchain-groq langchain_openai
%pip install -q langchain-huggingface
%pip install -qU langchain-qdrant
%pip install -qU qdrant_client
%pip install -qU langchain-core
%pip install -qU langchain-hub

In [ ]:
import getpass
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sklearn.cluster import KMeans
from typing import Any
from collections import defaultdict
import numpy as np
from tiktoken import Encoding, encoding_for_model, get_encoding
from statistics import mean

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

# RAG

In [ ]:
#LLMs and embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
llm_drafter = ChatOpenAI(model="gpt-4o-mini", logprobs=True)
llm_verifier = ChatOpenAI(model="gpt-4o", logprobs=True)

In [ ]:
#Vector store
qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

# Specify your collection name and query vector
collection_name = ""

### Multi perspective sampling

In [ ]:
def multi_perspective_sampling(
    k: int, retrieved_points: list[models.ScoredPoint], seed: int = 1399
) -> list[list[str]]:
    # Generate clusters
    print(f"Finding {k} clusters.")
    algo: Any = KMeans(n_clusters=k, random_state=seed)
    _vectors = [point.vector for point in retrieved_points]
    clusters: list[int] = algo.fit_predict(X=_vectors)

    # Unique clusters
    unique_clusters: set[int] = set(clusters)

    # Create a dictionary with the members of each cluster
    cluster_dict: defaultdict[int, list[int | None]] = defaultdict(list)
    for index, cluster in enumerate(clusters):
        cluster_dict[cluster].append(index)
    print(f"Clusters distribution: {dict(cluster_dict)}")

    # M subsets
    m: int = min(len(indices) for indices in cluster_dict.values())
    print(f"{m} document subsets will be created.")

    np.random.seed(seed=seed)
    subsets: list[list[str]] = []

    for _ in range(m):
        subset: list[int] = []
        for cluster in unique_clusters:
            chosen_element: int = np.random.choice(cluster_dict[cluster])
            subset.append(chosen_element)
            cluster_dict[cluster].remove(chosen_element)
        subset_documents = [
            retrieved_points[idx].payload.get("page_content") for idx in subset
        ]
        subsets.append(subset_documents)

    return subsets

### RAG drafting

In [ ]:
from openai import AsyncOpenAI


rag_drafting_prompt: str = """Trả lời câu hỏi dựa trên các văn bản được cung cấp. Đồng thời cung cấp lý do đưa ra câu trả lời của bạn.
## Câu hỏi: {query}

## Văn bản cung cấp: {evidence}"""


class RagDraftingResponse(BaseModel):
    rationale: str = Field(description="Lý do đưa ra câu trả lời.")
    response: str = Field(description="Câu trả lời dựa trên các băn bản được cung cấp.")


async def rag_drafting_generator(
    client: AsyncOpenAI,
    model_name: str,
    query: str,
    evidence: str,
    **kwargs,
) -> tuple[RagDraftingResponse, float]:
    completion: Any = await client.beta.chat.completions.parse(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": rag_drafting_prompt.format(
                    instruction=query, evidence=evidence
                ),
            }
        ],
        response_format=RagDraftingResponse,
        temperature=0.0,
        logprobs=True,
        max_tokens=512,
        **kwargs,
    )
    return (
        completion.choices[0].message.parsed,
        np.exp(mean(token.logprob for token in completion.choices[0].logprobs.content)),
    )

### RAG verifier

In [ ]:
rag_verifier_prompt: str = """## Câu hỏi: {query}

## Câu trả lời: {response}

## Lý do: {rationale}

Lý do có đủ tốt để hỗ trợ cho câu trả lời không? (Có hoặc không)"""


async def rag_verifier_generator(
    client: AsyncOpenAI,
    model_name: str,
    instruction: str,
    evidence: str,
    response: str,
    rationale: str,
    **kwargs,
) -> tuple[Any, float]:
    encoder: Encoding = encoding_for_model(model_name=model_name)
    completion: Any = await client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": rag_verifier_prompt.format(
                    instruction=instruction,
                    evidence=evidence,
                    response=response,
                    rationale=rationale,
                ),
            }
        ],
        temperature=0.0,
        logprobs=True,
        max_tokens=2,
        **kwargs,
    )
    response: str = completion.choices[0].message.content
    cond: bool = encoder.encode(text=response.lower()) == encoder.encode(text="yes")
    p_yes: float = (
        np.exp(mean(token.logprob for token in completion.choices[0].logprobs.content))
        if cond
        else 0.0
    )  # Naive

    return (response, p_yes)

### End - to - end

In [ ]:
import asyncio
from time import perf_counter
from venv import logger

from qdrant_client import AsyncQdrantClient

async def speculative_rag(
    query: str,
    embedding_model: str,
    collection_name: str,
    k: int,
    seed: int,
    client: AsyncOpenAI,
    qdrant_client: AsyncQdrantClient,
    m_drafter: str,
    m_verifier: str,
) -> str:
    _start = perf_counter()

    # Generate query vector embedding
    logger.info("Generating query vector...")
    _now: float = perf_counter()
    query_vector: Any = await client.embeddings.create(
        input=query, model=embedding_model
    )
    query_vector: list[float] = query_vector.data[0].embedding
    logger.info("Query vector generated in {s:.4f} seconds.", s=perf_counter() - _now)

    # Fetching relevant documents
    logger.info("Fetching relevant documents...")
    _now: float = perf_counter()
    out: list[models.ScoredPoint] = await qdrant_client.search(
        collection_name=collection_name, query_vector=query_vector, with_vectors=True
    )
    logger.info("Documents retrieved in {s:.4f} seconds.", s=perf_counter() - _now)

    # Multi Perspective Sampling
    logger.info("Doing Multi Perspective Sampling...")
    _now: float = perf_counter()
    sampled_docs: list[list[str]] = multi_perspective_sampling(
        k=k, retrieved_points=out, seed=seed
    )
    logger.info(
        "Multi Perspective Sampling done in {s:.4f} seconds.", s=perf_counter() - _now
    )

    # RAG Drafting
    logger.info("Doing RAG Drafting...")
    _now: float = perf_counter()
    rag_drafts: list[tuple[RagDraftingResponse, float]] = await asyncio.gather(
        *[
            rag_drafting_generator(
                client=client,
                model_name=m_drafter,
                query=query,
                evidence="\n".join(
                    [f"[{idx}] {doc}" for idx, doc in enumerate(subset, start=1)]
                ),
            )
            for subset in sampled_docs
        ]
    )
    logger.info("RAG Drafting done in {s:.4f} seconds.", s=perf_counter() - _now)

    # RAG Verifier
    logger.info("Doing RAG Verification...")
    _now: float = perf_counter()
    rag_verifications: list[tuple[str, float]] = await asyncio.gather(
        *[
            rag_verifier_generator(
                client=client,
                model_name=m_verifier,
                instruction=query,
                evidence="\n".join(
                    [f"[{idx}] {doc}" for idx, doc in enumerate(subset, start=1)]
                ),
                response=rag_drafting_response.response,
                rationale=rag_drafting_response.rationale,
            )
            for subset, (rag_drafting_response, _) in zip(sampled_docs, rag_drafts)
        ]
    )
    logger.info("RAG Verification done in {s:.4f} seconds.", s=perf_counter() - _now)

    best_answer: int = np.argmax(
        p_draft * p_self
        for (_, p_draft), (_, p_self) in zip(rag_drafts, rag_verifications)
    )
    logger.info("Entire process done in {s:.4f} seconds.", s=perf_counter() - _start)
    print(f"\nQuestion:\n ------ \n{query}\n\n")
    print(f"Response:\n ------ \n{rag_drafts[best_answer][0].response}")
    return rag_drafts[best_answer][0].response